In [1]:
pip install --upgrade tensorflow

     |████████████████████████████████| 421.8MB 36kB/s 
     |████████████████████████████████| 450kB 42.8MB/s 
     |████████████████████████████████| 3.8MB 50.8MB/s 
     |████████████████████████████████| 81kB 11.8MB/s 
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstal

In [1]:
pip install konlpy

     |████████████████████████████████| 19.4MB 164kB/s 
     |████████████████████████████████| 2.3MB 40.5MB/s 
     |████████████████████████████████| 92kB 13.7MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
import tensorflow as tf

print(tf.__version__)

2.1.0


In [0]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from konlpy.tag import Kkma
import time
from tensorflow.keras.layers import *
from tensorflow.keras import Model

BATCH_SIZE = 128
embedding_dim = 300
units = 128

data = pd.read_csv("data_new7.csv",encoding = 'cp949')

input_data = data.iloc[:,0].to_list()
output_data = data.iloc[:,1].to_list()

In [0]:
input_vocab,output_vocab = set(),set()
input_max_len = 0
output_max_len = 0
kkma = Kkma()
morphs_input_sentence = []
morphs_output_sentence = []

In [0]:
for input_sentence,output_sentence in zip(input_data,output_data):
    input_sentence = kkma.morphs(input_sentence)
    output_sentence = kkma.morphs(output_sentence)
    
    morphs_input_sentence.append(input_sentence)
    morphs_output_sentence.append(output_sentence)
    
    input_vocab.update(input_sentence)
    output_vocab.update(output_sentence)
    
    input_steplen = len(input_sentence)
    output_steplen = len(output_sentence)
    
    input_max_len = input_max_len if input_max_len > input_steplen else input_steplen
    output_max_len = output_max_len if output_max_len > output_steplen else output_steplen

In [0]:
input_vocab = ['<p>', '<start>', '<end>'] + list(input_vocab)
output_vocab = ['<p>', '<start>', '<end>'] + list(output_vocab)

input_vocab_size = len(input_vocab)
output_vocab_size = len(output_vocab)

dic_input_vocab = {word:index for index,word in enumerate(input_vocab)}
dic_output_vocab = {word:index for index,word in enumerate(output_vocab)}

In [0]:
input_tokens = []
output_tokens = []
for i,(inp_sen, targ_sen) in enumerate(zip(morphs_input_sentence, morphs_output_sentence)):
    s_input_token = [dic_input_vocab['<start>']] + [dic_input_vocab[word] for word in inp_sen] + [dic_output_vocab['<end>']]
    s_output_token = [dic_output_vocab['<start>']] + [dic_output_vocab[word] for word in targ_sen] + [dic_output_vocab["<end>"]]
    
    input_tokens.append(s_input_token)
    output_tokens.append(s_output_token)

input_tokens = pad_sequences(input_tokens, input_max_len, padding = 'post')
output_tokens = pad_sequences(output_tokens, output_max_len, padding = 'post')

In [28]:
print(s_input_token)

[1, 21199, 1094, 5822, 395, 13707, 12579, 19500, 4049, 18942, 18911, 8200, 10628, 18942, 10442, 15970, 15384, 19756, 9404, 10141, 14206, 10733, 11048, 2]


In [0]:
def preprocess(sentence, vocab = dic_input_vocab, max_sen_len = input_max_len):
    sen = sentence
    sen = kkma.morphs(sen)
    print(sen)
    sen = [[dic_input_vocab['<start>']] + [dic_input_vocab[word] for word in sen] + [dic_input_vocab["<end>"]]]
    sen = pad_sequences(sen, max_sen_len,padding = 'post')
    return sen

In [0]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, gru_hidden, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.gru_hidden = gru_hidden
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.gru_hidden,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.gru_hidden))

In [0]:
encoder = Encoder(len(input_vocab), embedding_dim = embedding_dim, gru_hidden = units, batch_sz = BATCH_SIZE)

In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self,units):
        super(BahdanauAttention,self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, query,values):
        #query => encoder hidden
        #values => decoder hidden
        hidden_with_time_axis = tf.expand_dims(query, 1)
        # hidden_with_time_axis의 shape은 (batch_size, 1, hidden_size)이다.
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))
        # score shape => (batch_size, seq_len, 1)
        attention_weights = tf.nn.softmax(score, axis = 1)
        context_vector = attention_weights*values
        context_vector = tf.reduce_sum(context_vector, axis = 1)
        
        return context_vector, attention_weights

In [0]:
class Decoder(tf.keras.Model):
    def __init__(self,vocab_size, embedding_dim, gru_hidden, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.gru_hidden = gru_hidden
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.gru_hidden,
                                      return_sequences=True,
                                      return_state = True,
                                      recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.attention = BahdanauAttention(self.gru_hidden)
        
    def call(self, x, gru_hidden, enc_output):
        context_vector,attention_weights = self.attention(gru_hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector,1),x], axis = -1)
        output,state = self.gru(x)
        output = tf.reshape(output,(-1,output.shape[2]))
        x = self.fc(output)
        return x, state, attention_weights

In [0]:
decoder = Decoder(len(output_vocab), embedding_dim, units, BATCH_SIZE)

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction = 'none')
def loss_function(real,pred):
    mask = tf.math.logical_not(tf.math.equal(real,0))
    loss_ = loss_object(real,pred)
    
    mask = tf.cast(mask,dtype = loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [0]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([dic_output_vocab['<start>']] * BATCH_SIZE, 1)

        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:, t], predictions)

            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [0]:
steps_per_epoch = len(input_tokens)//BATCH_SIZE

In [38]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for batch in range(len(input_tokens)//BATCH_SIZE):
        batch_input = input_tokens[batch*BATCH_SIZE: (batch+1)*BATCH_SIZE]
        batch_output = output_tokens[batch*BATCH_SIZE: (batch+1)*BATCH_SIZE]

        batch_loss = train_step(batch_input, batch_output, enc_hidden)
        total_loss += batch_loss

        if batch % 50 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 2.1359
Epoch 1 Batch 50 Loss 2.2156
Epoch 1 Batch 100 Loss 2.2058
Epoch 1 Batch 150 Loss 2.1381
Epoch 1 Batch 200 Loss 1.0275
Epoch 1 Batch 250 Loss 1.0582
Epoch 1 Batch 300 Loss 1.4102
Epoch 1 Batch 350 Loss 1.0876
Epoch 1 Batch 400 Loss 1.0844
Epoch 1 Batch 450 Loss 1.1939
Epoch 1 Batch 500 Loss 1.1252
Epoch 1 Batch 550 Loss 1.0619
Epoch 1 Batch 600 Loss 1.1160
Epoch 1 Batch 650 Loss 0.9627
Epoch 1 Batch 700 Loss 0.9948
Epoch 1 Batch 750 Loss 1.0179
Epoch 1 Loss 1.3604
Time taken for 1 epoch 307.67921137809753 sec

Epoch 2 Batch 0 Loss 1.1873
Epoch 2 Batch 50 Loss 1.9099
Epoch 2 Batch 100 Loss 1.8324
Epoch 2 Batch 150 Loss 1.7557
Epoch 2 Batch 200 Loss 0.8695
Epoch 2 Batch 250 Loss 0.8747
Epoch 2 Batch 300 Loss 1.1992
Epoch 2 Batch 350 Loss 0.9305
Epoch 2 Batch 400 Loss 0.9268
Epoch 2 Batch 450 Loss 1.0254
Epoch 2 Batch 500 Loss 0.9724
Epoch 2 Batch 550 Loss 0.9196
Epoch 2 Batch 600 Loss 0.9734
Epoch 2 Batch 650 Loss 0.8522
Epoch 2 Batch 700 Loss 0.8846
Epoch 2 B

In [0]:
def evaluate(sentence):
    inp = preprocess(sentence)

    inputs = tf.convert_to_tensor(inp)

    result = ''
    
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([dic_input_vocab['<start>']], 0)

    for t in range(output_max_len):
        predictions, dec_hidden, attention_weights = decoder(dec_input, 
                                                             dec_hidden,
                                                             enc_out)
        attention_weights = tf.reshape(attention_weights, (-1,))
        

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += output_vocab[predicted_id] + ' '
        
        if output_vocab[predicted_id] == '<end>':
            return result, sentence
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

In [0]:
def translate(sentence):
    result, sentence = evaluate(sentence)
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

In [50]:
translate('재미없는 기계학습')

['재미없', '는', '기계', '학습']
Input: 재미없는 기계학습
Predicted translation: 재미없 는 기계 타 고 <end> 


In [0]:
input_data[0]